In [ ]:
!pip install -e /dss/dsshome1/04/di93zer/git/cellnet --no-deps

In [ ]:
import os
import seaborn as sns
import torch

from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor, TQDMProgressBar
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.utilities.model_summary import ModelSummary
from lightning.pytorch import seed_everything

In [ ]:
torch.set_float32_matmul_precision('high')

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload
from cellnet.estimators import EstimatorCellTypeClassifier

# Init model

In [ ]:
# config parameters
MODEL = 'cxg_2023_05_15_linear'
CHECKPOINT_PATH = os.path.join('/mnt/dssfs02/tb_logs', MODEL)
LOGS_PATH = os.path.join('/mnt/dssfs02/tb_logs', MODEL)
DATA_PATH = '/mnt/dssmcmlfs01/merlin_cxg_2023_05_15_sf-log1p_subsample_15'
SEED = 1


estim = EstimatorCellTypeClassifier(DATA_PATH)
seed_everything(SEED)
estim.init_datamodule(batch_size=2048)
estim.init_trainer(
    trainer_kwargs={
        'max_epochs': 12,
        'default_root_dir': CHECKPOINT_PATH,
        'accelerator': 'gpu',
        'devices': 1,
        'num_sanity_val_steps': 0,
        'check_val_every_n_epoch': 1,
        'logger': [TensorBoardLogger(LOGS_PATH, name='default')],
        'log_every_n_steps': 100,
        'detect_anomaly': False,
        'enable_progress_bar': True,
        'enable_model_summary': False,
        'enable_checkpointing': True,
        'callbacks': [
            TQDMProgressBar(refresh_rate=50),
            LearningRateMonitor(logging_interval='step'),
            ModelCheckpoint(filename='val_f1_macro_{epoch}_{val_f1_macro:.3f}', monitor='val_f1_macro', mode='max',
                            every_n_epochs=1, save_top_k=2),
            ModelCheckpoint(filename='val_loss_{epoch}_{val_loss:.3f}', monitor='val_loss', mode='min',
                            every_n_epochs=1, save_top_k=2)
        ],
    }
)
estim.init_model(
    model_type='linear',
    model_kwargs={
        'learning_rate': 0.0005,
        'weight_decay': 0.05,
        'optimizer': torch.optim.AdamW,
        'lr_scheduler': torch.optim.lr_scheduler.StepLR,
        'lr_scheduler_kwargs': {
            'step_size': 3,
            'gamma': 0.9,
            'verbose': True
        },
    },
)
print(ModelSummary(estim.model))

# Find learning rate

In [ ]:
lr_find_res = estim.find_lr(lr_find_kwargs={'early_stop_threshold': 10., 'min_lr': 1e-8, 'max_lr': 10., 'num_training': 120})

In [ ]:
ax = sns.lineplot(x=lr_find_res[1]['lr'], y=lr_find_res[1]['loss'])
ax.set_xscale('log')
ax.set_ylim(2., top=9.)
ax.set_xlim(1e-6, 10.)
print(f'Suggested learning rate: {lr_find_res[0]}')

# Fit model

In [ ]:
estim.train()